## Spacy-Transformer Roberta-Base Model

In [ ]:
!pip install spacy_transformers  #Needed for spacy transformer
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 1.1 MB 51.4 MB/s 
     |████████████████████████████████| 4.7 MB 58.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.5 MB/s 
     |████████████████████████████████| 182 kB 71.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## For Docx Files

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
!nvidia-smi

Tue Nov 22 17:41:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |    336MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json

In [ ]:
cv_data=json.load(open('/content/drive/MyDrive/Training/train_data.json','r'))

In [ ]:
len(cv_data)  #there are 200 resume which we are gonna use for training purporse. 70% Training, 30% testing

200

In [ ]:
# creates config.cfg file
!python -m spacy init fill-config /content/drive/MyDrive/Training/base_config.cfg /content/drive/MyDrive/Training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## We are now changing instances of the config file from 20000 to 2000. 
## max_steps = 2000, line 94


In [ ]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db=DocBin()

#Training data,which is in JSON format, is in the format of 1st text, then its annotation
                                 
  for text,annot in tqdm(data):    #tqdm to check the progress
    doc=nlp.make_doc(text)
    annot=annot['entities'] # entities/entity_labels are skills, organization, university, passing-year,degree

    ents=[]
    entity_indices=[]
    
    for start,end,label in annot:  #annot entities have 3 labels in our tarining data                                         
                                                # start character index, end character index & label
      skip_entity=False      # sometimes data gets annotated wrongly as it is manually written in CV hence we do it false 

    # there are overlaps of entity names in our data, hence to make unique start,end we are writing this
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity= True   # to skip overlapping entities
          break

        if skip_entity==True:
          continue

        entity_indices = entity_indices + list(range(start,end)) # updating entities list with unique entities
    
    # Here, we are trying to get the particular piece of the document defined in each of the entities with the
    # help of start and end character index
    # if that piece is present, it will fetch it.
    # if not present, it will continue to next entity.
        try:
          span = doc.char_span(start,end,label=label,alignment_mode='strict')  #mode can be strict or contract
        except:
          continue

        # span is None when start and end values are same, then we ll put here an error data
        if span is None:
          err_data =  str([start,end]) + "  " + str(text) + "\n"
          file.write(err_data)    # later we can check what are the resume files that are throwing the errors 

        else:
          ents.append(span) #if span is Not None and No errors, then we add it to ents list
    
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  
  return db  #return DocBin object

## **Splitting Training_data into train-test split for Modelling**

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)  #140 resumes for training , 60 for testing

(140, 60)

In [ ]:
# creating file object for error file

file=open('error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')  #training data that we would be storing into train_data.spacy

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')   # making spacy doc object for training and testing data

file.close()

100%|██████████| 60/60 [00:00<00:00, 111.01it/s]


### IN error.txt file we can see each of the CVs have some errors in their entities. These are highlighted here, overlapping entites.

# **Training the Model**

https://spacy.io/usage/training#basics

We can use this command to train our model:

 python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
!python -m spacy train /content/drive/MyDrive/Training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-22 18:25:01,154] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-22 18:25:01,164] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-11-22 18:25:01,168] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-22 18:25:01,169] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

In [ ]:
import pickle

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/PDF_Resume/output/model-best')

In [ ]:
pickle.dump(nlp, open('ner_model.pkl', 'wb'))